In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import requests
import scipy as sp
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, roc_curve, auc, confusion_matrix

In [3]:
icd10_df = pd.read_csv('VEPwithGB_icd10_0323.csv')
icd10_df

/Users/joannahench/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,9,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Location,Allele,Protein_position,SYMBOL,Amino_acids,locus,SIFT,PolyPhen,BayesDel_addAF_score,BayesDel_noAF_score,...,coding_description,category,AC,AF,BETA,SE,AF.Cases,AF.Controls,Pvalue,vcf
0,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,0.431260,0.53696,0.003472,0.002413,0.421890,11 113412715 . T C
1,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,-0.246980,0.61625,0.001845,0.002416,0.688580,11 113412715 . T C
2,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,1.305800,1.07300,0.005556,0.002413,0.223580,11 113412715 . T C
3,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,-0.141640,0.27161,0.002068,0.002417,0.602020,11 113412715 . T C
4,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,0.071904,0.52037,0.002584,0.002415,0.890100,11 113412715 . T C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654733,X:114907411-114907411,C,458,HTR2C,V/A,chrX:114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.007700,7.82990,0.000000,0.000004,0.897600,X 114907411 . T C
654734,X:114907411-114907411,C,458,HTR2C,V/A,chrX:114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,5.129800,2.33040,0.000049,0.000003,0.027721,X 114907411 . T C
654735,X:114907411-114907411,C,458,HTR2C,V/A,chrX:114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.015900,4.91580,0.000000,0.000004,0.836280,X 114907411 . T C
654736,X:114907411-114907411,C,458,HTR2C,V/A,chrX:114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.022800,3.18950,0.000000,0.000004,0.748460,X 114907411 . T C


In [4]:
icd10_df['mutation_from'] = icd10_df['Amino_acids'].str.split('/',expand=True)[0]
icd10_df['mutation_to'] = icd10_df['Amino_acids'].str.split('/',expand=True)[1]
icd10_df

,Location,Allele,Protein_position,SYMBOL,Amino_acids,locus,SIFT,PolyPhen,BayesDel_addAF_score,BayesDel_noAF_score,...,AC,AF,BETA,SE,AF.Cases,AF.Controls,Pvalue,vcf,mutation_from,mutation_to
0,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,1907,0.002415,0.431260,0.53696,0.003472,0.002413,0.421890,11 113412715 . T C,K,E
1,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,1907,0.002415,-0.246980,0.61625,0.001845,0.002416,0.688580,11 113412715 . T C,K,E
2,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,1907,0.002415,1.305800,1.07300,0.005556,0.002413,0.223580,11 113412715 . T C,K,E
3,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,1907,0.002415,-0.141640,0.27161,0.002068,0.002417,0.602020,11 113412715 . T C,K,E
4,11:113412715-113412715,C,327,DRD2,K/E,chr11:113412715,tolerated(0.27),benign(0.031),-0.260005,-0.140988,...,1907,0.002415,0.071904,0.52037,0.002584,0.002415,0.890100,11 113412715 . T C,K,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654733,X:114907411-114907411,C,458,HTR2C,V/A,chrX:114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,3,0.000004,-1.007700,7.82990,0.000000,0.000004,0.897600,X 114907411 . T C,V,A
654734,X:114907411-114907411,C,458,HTR2C,V/A,chrX:114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,3,0.000004,5.129800,2.33040,0.000049,0.000003,0.027721,X 114907411 . T C,V,A
654735,X:114907411-114907411,C,458,HTR2C,V/A,chrX:114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,3,0.000004,-1.015900,4.91580,0.000000,0.000004,0.836280,X 114907411 . T C,V,A
654736,X:114907411-114907411,C,458,HTR2C,V/A,chrX:114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,3,0.000004,-1.022800,3.18950,0.000000,0.000004,0.748460,X 114907411 . T C,V,A


In [5]:
dyna_df = pd.read_csv("Genebass_dynamut2_out.csv")
dyna_df = dyna_df.rename(columns={'prediction':'Dynamut2'})

dyna_df

,mutation_from,Protein_position,mutation_to,mutation,chain,Dynamut2,Uniprot,protein,SYMBOL
0,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2
1,E,180,Q,E180Q,A,-0.23,P07550,adrb2_human,ADRB2
2,N,15,D,N15D,A,-0.11,P07550,adrb2_human,ADRB2
3,Q,3,K,Q3K,A,-0.07,P07550,adrb2_human,ADRB2
4,R,343,C,R343C,A,0.18,P07550,adrb2_human,ADRB2
...,...,...,...,...,...,...,...,...,...
15479,R,349,Q,R349Q,A,-0.69,Q9Y5N1,hrh3_human,HRH3
15480,P,271,L,P271L,A,-0.30,Q9Y5N1,hrh3_human,HRH3
15481,S,135,L,S135L,A,-0.40,Q9Y5N1,hrh3_human,HRH3
15482,W,196,C,W196C,A,0.16,Q9Y5N1,hrh3_human,HRH3


In [6]:
gb_with_dyna = pd.merge(dyna_df,icd10_df,on=['mutation_from','Protein_position','mutation_to','SYMBOL'])
gb_with_dyna

,mutation_from,Protein_position,mutation_to,mutation,chain,Dynamut2,Uniprot,protein,SYMBOL,Location,...,coding_description,category,AC,AF,BETA,SE,AF.Cases,AF.Controls,Pvalue,vcf
0,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1,0.000001,-1.0004,57.3580,0.0,0.000001,0.98608,5 148827523 . A G
1,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1,0.000001,-1.0003,58.7630,0.0,0.000001,0.98642,5 148827523 . A G
2,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1,0.000001,-1.0002,102.7300,0.0,0.000001,0.99223,5 148827523 . A G
3,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1,0.000001,-1.0032,19.1680,0.0,0.000001,0.95826,5 148827523 . A G
4,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1,0.000001,-1.0008,38.3230,0.0,0.000001,0.97916,5 148827523 . A G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654418,H,387,Y,H387Y,A,1.17,Q9Y5N1,hrh3_human,HRH3,20:62216185-62216185,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.0365,4.1876,0.0,0.000004,0.80451,20 62216185 . G A
654419,H,387,Y,H387Y,A,1.17,Q9Y5N1,hrh3_human,HRH3,20:62216185-62216185,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.0365,3.4088,0.0,0.000004,0.76108,20 62216185 . G A
654420,H,387,Y,H387Y,A,1.17,Q9Y5N1,hrh3_human,HRH3,20:62216185-62216185,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.0163,4.7195,0.0,0.000004,0.82950,20 62216185 . G A
654421,H,387,Y,H387Y,A,1.17,Q9Y5N1,hrh3_human,HRH3,20:62216185-62216185,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.0252,3.9932,0.0,0.000004,0.79739,20 62216185 . G A


In [7]:
gb_with_dyna.to_csv('Genebass_with_Dynamut2_df.csv',index=False)

In [19]:
# ddg into 3 classifications: (neg infinity,-1), [-1,1],(1,pos infinity)
gb_with_dyna['Dynamut2_group'] = gb_with_dyna['prediction'].apply(lambda x: 1 if x < -1 else (1 if x > 1 else 0))
gb_with_dyna['GB_group'] = gb_with_dyna.apply(lambda row: 1 if (row['BETA'] > 2 and row['Pvalue'] < 0.05) else 0, axis=1)
gb_with_dyna


,mutation_from,mutation_pos,mutation_to,mutation,chain,prediction,Uniprot,protein,SYMBOL,Location,...,AC,AF,BETA,SE,AF.Cases,AF.Controls,Pvalue,vcf,Dynamut2_group,GB_group
0,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,1,0.000001,-1.0004,57.3580,0.0,0.000001,0.98608,5 148827523 . A G,0,0
1,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,1,0.000001,-1.0003,58.7630,0.0,0.000001,0.98642,5 148827523 . A G,0,0
2,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,1,0.000001,-1.0002,102.7300,0.0,0.000001,0.99223,5 148827523 . A G,0,0
3,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,1,0.000001,-1.0032,19.1680,0.0,0.000001,0.95826,5 148827523 . A G,0,0
4,Q,231,R,Q231R,A,-0.49,P07550,adrb2_human,ADRB2,5:148827523-148827523,...,1,0.000001,-1.0008,38.3230,0.0,0.000001,0.97916,5 148827523 . A G,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477607,V,66,L,V66L,A,-0.19,Q9H3N8,hrh4_human,HRH4,18:24468790-24468790,...,2,0.000003,-1.0111,7.2594,0.0,0.000003,0.88923,18 24468790 . G T,0,0
477608,V,66,L,V66L,A,-0.19,Q9H3N8,hrh4_human,HRH4,18:24468790-24468790,...,2,0.000003,-1.0242,4.6822,0.0,0.000003,0.82685,18 24468790 . G T,0,0
477609,V,66,L,V66L,A,-0.19,Q9H3N8,hrh4_human,HRH4,18:24468790-24468790,...,2,0.000003,-1.0179,5.4836,0.0,0.000003,0.85274,18 24468790 . G T,0,0
477610,V,66,L,V66L,A,-0.19,Q9H3N8,hrh4_human,HRH4,18:24468790-24468790,...,2,0.000003,-1.0387,3.8658,0.0,0.000003,0.78816,18 24468790 . G T,0,0


In [20]:
true_labels = gb_with_dyna['GB_group']
predicted_labels = gb_with_dyna['Dynamut2_group']
# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels).ravel()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate specificity and sensitivity
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)


# Calculate precision, recall and F1-score
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# calculate FPR and TPR
fpr, tpr, thresholds = roc_curve(true_labels, predicted_labels)

# calculate AUCROC
auc_roc = auc(fpr, tpr)



print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("AUCROC score: {:.3f}".format(auc_roc))


Specificity: 0.8319691247979246
Sensitivity: 0.1557528671846097
Accuracy: 0.8204881786889776
Precision: 0.015757167452653643
F1-score: 0.028619013629720264
AUCROC score: 0.494
